# Step 2-3. Analyze Results of a Hyperparameter Tuning job (Optional)

Once you have completed a tuning job, (or even while the job is still running) you can use this notebook to analyze the results to understand how each hyperparameter effects the quality of the model.

---
## Set up the environment
To start the analysis, you must pick the name of the hyperparameter tuning job.

In [2]:
import boto3
import sagemaker
import os

region = boto3.Session().region_name
sage_client = boto3.Session().client('sagemaker')

tuning_job_name = 'sagemaker-xgboost-191010-1249'
print(sagemaker.__version__)

1.37.2


## Track hyperparameter tuning job progress
After you launch a tuning job, you can see its progress by calling describe_tuning_job API. The output from describe-tuning-job is a JSON object that contains information about the current state of the tuning job. You can call list_training_jobs_for_tuning_job to see a detailed list of the training jobs that the tuning job launched.

In [3]:
# run this cell to check current status of hyperparameter tuning job
tuning_job_result = sage_client.describe_hyper_parameter_tuning_job(HyperParameterTuningJobName=tuning_job_name)

status = tuning_job_result['HyperParameterTuningJobStatus']
if status != 'Completed':
    print('Reminder: the tuning job has not been completed.')
    
job_count = tuning_job_result['TrainingJobStatusCounters']['Completed']
print("%d training jobs have completed" % job_count)
    
is_minimize = (tuning_job_result['HyperParameterTuningJobConfig']['HyperParameterTuningJobObjective']['Type'] != 'Maximize')
objective_name = tuning_job_result['HyperParameterTuningJobConfig']['HyperParameterTuningJobObjective']['MetricName']

150 training jobs have completed


In [4]:
from pprint import pprint
if tuning_job_result.get('BestTrainingJob',None):
    print("Best model found so far:")
    pprint(tuning_job_result['BestTrainingJob'])
else:
    print("No training jobs have reported results yet.")

Best model found so far:
{'CreationTime': datetime.datetime(2019, 10, 10, 13, 19, 12, tzinfo=tzlocal()),
 'FinalHyperParameterTuningJobObjectiveMetric': {'MetricName': 'validation:auc',
                                                 'Value': 0.986644983291626},
 'ObjectiveStatus': 'Succeeded',
 'TrainingEndTime': datetime.datetime(2019, 10, 10, 13, 21, 48, tzinfo=tzlocal()),
 'TrainingJobArn': 'arn:aws:sagemaker:us-west-2:143656149352:training-job/sagemaker-xgboost-191010-1249-054-2f999db2',
 'TrainingJobName': 'sagemaker-xgboost-191010-1249-054-2f999db2',
 'TrainingJobStatus': 'Completed',
 'TrainingStartTime': datetime.datetime(2019, 10, 10, 13, 21, tzinfo=tzlocal()),
 'TunedHyperParameters': {'alpha': '0.06659224220274589',
                          'colsample_bytree': '0.5166123357270381',
                          'eta': '0.09255858734037499',
                          'gamma': '0',
                          'lambda': '0.029680038043396386',
                          'min_child_

## Fetch all results as DataFrame
We can list hyperparameters and objective metrics of all training jobs and pick up the training job with the best objective metric.

In [5]:
import pandas as pd

tuner = sagemaker.HyperparameterTuningJobAnalytics(tuning_job_name)

full_df = tuner.dataframe()

if len(full_df) > 0:
    df = full_df[full_df['FinalObjectiveValue'] > -float('inf')]
    if len(df) > 0:
        df = df.sort_values('FinalObjectiveValue', ascending=is_minimize)
        print("Number of training jobs with valid objective: %d" % len(df))
        print({"lowest":min(df['FinalObjectiveValue']),"highest": max(df['FinalObjectiveValue'])})
        pd.set_option('display.max_colwidth', -1)  # Don't truncate TrainingJobName        
    else:
        print("No training jobs have reported valid results yet.")
        
df

Number of training jobs with valid objective: 150
{'lowest': 0.951662003993988, 'highest': 0.986644983291626}


,FinalObjectiveValue,TrainingElapsedTimeSeconds,TrainingEndTime,TrainingJobName,TrainingJobStatus,TrainingStartTime,alpha,colsample_bytree,eta,gamma,lambda,min_child_weight,subsample
96,0.986645,48.0,2019-10-10 13:21:48+00:00,sagemaker-xgboost-191010-1249-054-2f999db2,Completed,2019-10-10 13:21:00+00:00,0.066592,0.516612,0.092559,0.0,0.029680,15.0,0.763270
29,0.986321,42.0,2019-10-10 14:02:19+00:00,sagemaker-xgboost-191010-1249-121-17db0bdb,Completed,2019-10-10 14:01:37+00:00,0.162667,0.504637,0.060709,5.0,0.013792,11.0,0.646412
21,0.986157,50.0,2019-10-10 14:07:35+00:00,sagemaker-xgboost-191010-1249-129-78345252,Completed,2019-10-10 14:06:45+00:00,0.727776,0.526134,0.083504,3.0,0.015639,11.0,0.759163
47,0.985578,49.0,2019-10-10 13:51:18+00:00,sagemaker-xgboost-191010-1249-103-98b0e6eb,Completed,2019-10-10 13:50:29+00:00,0.182587,0.500792,0.054105,5.0,0.010000,14.0,0.717122
44,0.985222,51.0,2019-10-10 13:53:45+00:00,sagemaker-xgboost-191010-1249-106-645971a8,Completed,2019-10-10 13:52:54+00:00,0.195646,0.503792,0.053205,5.0,0.010628,14.0,0.714122
60,0.985168,44.0,2019-10-10 13:42:48+00:00,sagemaker-xgboost-191010-1249-090-cdda01fd,Completed,2019-10-10 13:42:04+00:00,0.271713,0.503298,0.080852,5.0,0.015526,12.0,0.688688
63,0.985012,51.0,2019-10-10 13:41:54+00:00,sagemaker-xgboost-191010-1249-087-f3a63a7a,Completed,2019-10-10 13:41:03+00:00,0.253577,0.500298,0.081752,5.0,0.014490,12.0,0.691688
9,0.984934,48.0,2019-10-10 14:14:15+00:00,sagemaker-xgboost-191010-1249-141-0c2c87ce,Completed,2019-10-10 14:13:27+00:00,1.154612,0.531413,0.094928,5.0,0.028105,10.0,0.762855
30,0.984873,50.0,2019-10-10 14:01:54+00:00,sagemaker-xgboost-191010-1249-120-03ee8ff0,Completed,2019-10-10 14:01:04+00:00,0.351600,0.527681,0.087698,5.0,0.013405,16.0,0.710045
102,0.984377,47.0,2019-10-10 13:18:45+00:00,sagemaker-xgboost-191010-1249-048-4cd21f2b,Completed,2019-10-10 13:17:58+00:00,0.066592,0.516612,0.094359,0.0,0.029680,15.0,0.763270


## See TuningJob results vs time
Next we will show how the objective metric changes over time, as the tuning job progresses.  For Bayesian strategy, you should expect to see a general trend towards better results, but this progress will not be steady as the algorithm needs to balance _exploration_ of new areas of parameter space against _exploitation_ of known good areas.  This can give you a sense of whether or not the number of training jobs is sufficient for the complexity of your search space.

In [6]:
import bokeh
import bokeh.io
bokeh.io.output_notebook()
from bokeh.plotting import figure, show
from bokeh.models import HoverTool

class HoverHelper():

    def __init__(self, tuning_analytics):
        self.tuner = tuning_analytics

    def hovertool(self):
        tooltips = [
            ("FinalObjectiveValue", "@FinalObjectiveValue"),
            ("TrainingJobName", "@TrainingJobName"),
        ]
        for k in self.tuner.tuning_ranges.keys():
            tooltips.append( (k, "@{%s}" % k) )

        ht = HoverTool(tooltips=tooltips)
        return ht

    def tools(self, standard_tools='pan,crosshair,wheel_zoom,zoom_in,zoom_out,undo,reset'):
        return [self.hovertool(), standard_tools]

hover = HoverHelper(tuner)

p = figure(plot_width=900, plot_height=400, tools=hover.tools(), x_axis_type='datetime')
p.circle(source=df, x='TrainingStartTime', y='FinalObjectiveValue')
show(p)

Loading BokehJS ...

## Analyze the correlation between objective metric and individual hyperparameters 
Now you have finished a tuning job, you may want to know the correlation between your objective metric and individual hyperparameters you've selected to tune. Having that insight will help you decide whether it makes sense to adjust search ranges for certain hyperparameters and start another tuning job. For exmaple, if you see a positive trend between objective metric and a numerical hyperparameter, you probably want to set a higher tuning range for that hyperparameter in your next tuning job.

The following cell draws a graph for each hyperparameter to show its correlation with your objective metric.

In [7]:
ranges = tuner.tuning_ranges
figures = []
for hp_name, hp_range in ranges.items():
    categorical_args = {}
    if hp_range.get('Values'):
        # This is marked as categorical.  Check if all options are actually numbers.
        def is_num(x):
            try:
                float(x)
                return 1
            except:
                return 0           
        vals = hp_range['Values']
        if sum([is_num(x) for x in vals]) == len(vals):
            # Bokeh has issues plotting a "categorical" range that's actually numeric, so plot as numeric
            print("Hyperparameter %s is tuned as categorical, but all values are numeric" % hp_name)
        else:
            # Set up extra options for plotting categoricals.  A bit tricky when they're actually numbers.
            categorical_args['x_range'] = vals

    # Now plot it
    p = figure(plot_width=500, plot_height=500, 
               title="Objective vs %s" % hp_name,
               tools=hover.tools(),
               x_axis_label=hp_name, y_axis_label=objective_name,
               **categorical_args)
    p.circle(source=df, x=hp_name, y='FinalObjectiveValue')
    figures.append(p)
show(bokeh.layouts.Column(*figures))